### Goals
- Ingest and wrangle nflverse data

### Steps
- Download fresh data from nflverse and save to disk
- Split data into smaller dimensions .e.g game info vs play info vs play analytics, etc.
- Store data to SQL for further processing

# <font color=teal>imports</font>
Most processing is performed in python code, and there's a python module to do everything here without manual

In [1]:
import logging

from src import configs
import os
import sys
sys.path.append(os.path.abspath("../src"))



Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)


In [2]:
from src.nflverse_transform_job import load_files
from src.pbp_fact import transform_pbp
from src.nflverse_reader_job import URLReader
from src.pbp_participation import transform_pbp_participation
from src.player_stats import transform_player_stats, merge_injuries
from src.player_injuries import prep_player_injuries
from src.player_stats import transform_players
from src.db_utils import load_dims_to_db


# <font color=teal>housekeeping</font>

In [3]:
LOAD_TO_DB = True
database_schema = 'controls'

# Get the logger
logger = configs.configure_logging("pbp_logger")
logger.setLevel(logging.INFO)

---

# <font color=teal>load and transform play by play datasets</font>

### <font color="#9370DB">load</font>

In [4]:

%%time
pbp = load_files(data_subdir='pbp')


2023-07-16 17:56:42,739 - INFO - Reading all files from pbp
2023-07-16 17:56:42,741 - INFO -   + Reading pbp_2019.parquet
2023-07-16 17:56:42,884 - INFO -   + Reading pbp_2018.parquet
2023-07-16 17:56:42,987 - INFO -   + Reading pbp_2022.parquet
2023-07-16 17:56:43,092 - INFO -   + Reading pbp_2021.parquet
2023-07-16 17:56:43,197 - INFO -   + Reading pbp_2017.parquet
2023-07-16 17:56:43,290 - INFO -   + Reading pbp_2016.parquet
2023-07-16 17:56:43,387 - INFO -   + Reading pbp_2020.parquet


CPU times: user 2.34 s, sys: 619 ms, total: 2.96 s
Wall time: 2.01 s


### <font color="#9370DB">transform</font>

In [5]:

%%time
datasets = transform_pbp(pbp)

2023-07-16 17:56:46,258 - INFO - Impute columns to 0
2023-07-16 17:56:46,423 - INFO - impute non binary pbp columns ...
2023-07-16 17:56:46,976 - INFO - Impute columns to 0
2023-07-16 17:56:47,612 - INFO - Impute columns to 0:00
2023-07-16 17:56:48,930 - INFO - Impute columns to NA
2023-07-16 17:56:54,356 - INFO - moving play_id to play_counter, and creating a joinable play_id key
2023-07-16 17:56:55,104 - INFO - Conform key actions like pass, rush, kickoff, etc. and add a single category field called actions... 
2023-07-16 17:57:06,734 - INFO - Validate actions dimension ...
2023-07-16 17:57:06,996 - INFO - Creating new drive dimension...
2023-07-16 17:57:07,063 - INFO - Validate drive_df dimension ...
2023-07-16 17:57:07,286 - INFO - Creating new situations dimension...
2023-07-16 17:57:07,349 - INFO - Validate situation_df dimension ...
2023-07-16 17:57:07,552 - INFO - Creating new metrics dimension...
2023-07-16 17:57:07,597 - INFO - Validate play_metrics_df dimension ...
2023-07-1

CPU times: user 22.4 s, sys: 6.45 s, total: 28.9 s
Wall time: 29.8 s


---

# <font color=teal>load and transform play by play participation datasets</font>

### <font color="#9370DB">load</font>

In [6]:
%%time
pbp_participation_df = load_files('pbp-participation')


2023-07-16 17:57:14,547 - INFO - Reading all files from pbp-participation
2023-07-16 17:57:14,548 - INFO -   + Reading pbp-participation_2019.parquet
2023-07-16 17:57:14,577 - INFO -   + Reading pbp-participation_2018.parquet
2023-07-16 17:57:14,599 - INFO -   + Reading pbp-participation_2017.parquet
2023-07-16 17:57:14,620 - INFO -   + Reading pbp-participation_2021.parquet
2023-07-16 17:57:14,643 - INFO -   + Reading pbp-participation_2020.parquet
2023-07-16 17:57:14,662 - INFO -   + Reading pbp-participation_2016.parquet
2023-07-16 17:57:14,682 - INFO -   + Reading pbp-participation_2022.parquet


CPU times: user 252 ms, sys: 93.6 ms, total: 346 ms
Wall time: 248 ms


### <font color="#9370DB">transform</font>

In [7]:
%%time
player_df, player_events_df = transform_pbp_participation(
    participation_df=pbp_participation_df,
    player_events=datasets['player_events'])

datasets.update({
    'player_participation': player_df,
    'player_events': player_events_df,
})

2023-07-16 17:57:14,798 - INFO - pbp_participation:  move play_id to a play_count column and create a unique play_id that can be used in joins...
2023-07-16 17:57:15,023 - INFO - Calculating defense and offense team names by player and play...
2023-07-16 17:57:17,549 - INFO - Exploding offensive players to their own dataset...
2023-07-16 17:57:19,011 - INFO - Exploding defense_players to their own dataset...
2023-07-16 17:57:27,167 - WARNING - combining offense and defense players - counts are incorrect 6903461 != 6902991


CPU times: user 14.4 s, sys: 1.71 s, total: 16.1 s
Wall time: 16.2 s


---

# <font color=teal>transform player injuries</font>

### <font color="#9370DB">load</font>

In [8]:
%%time
injuries_df = load_files('injuries')

2023-07-16 17:57:30,987 - INFO - Reading all files from injuries
2023-07-16 17:57:30,989 - INFO -   + Reading injuries_2017.parquet
2023-07-16 17:57:30,993 - INFO -   + Reading injuries_2021.parquet
2023-07-16 17:57:30,998 - INFO -   + Reading injuries_2020.parquet
2023-07-16 17:57:31,002 - INFO -   + Reading injuries_2016.parquet
2023-07-16 17:57:31,006 - INFO -   + Reading injuries_2022.parquet
2023-07-16 17:57:31,010 - INFO -   + Reading injuries_2019.parquet
2023-07-16 17:57:31,014 - INFO -   + Reading injuries_2018.parquet


CPU times: user 43.8 ms, sys: 8.91 ms, total: 52.7 ms
Wall time: 39.7 ms


### <font color="#9370DB">transform</font>

In [9]:
%%time
injuries_df = prep_player_injuries(injuries_df)

2023-07-16 17:57:31,030 - INFO - Prep injury data...
2023-07-16 17:57:31,030 - INFO - Conforming names (e.g. gsis_id -> player_id)
2023-07-16 17:57:31,039 - INFO - Merge sparse injury columns
2023-07-16 17:57:31,041 - INFO - Get best values for null report_statuses...
2023-07-16 17:57:31,104 - INFO - check that all positions are correct...


CPU times: user 75.6 ms, sys: 3.95 ms, total: 79.6 ms
Wall time: 81.6 ms


---

# <font color=teal>transform player stats</font>

In [10]:
%%time
stats_df = load_files('player-stats')
stats_df = transform_player_stats(stats_df)
stats_df = merge_injuries(player_stats=stats_df, player_injuries=injuries_df)

2023-07-16 17:57:31,114 - INFO - Reading all files from player-stats
2023-07-16 17:57:31,115 - INFO -   + Reading player-stats.parquet
2023-07-16 17:57:31,152 - INFO - fix specific player_stats: <function player_stats_fixes at 0x2c4601f70>..
2023-07-16 17:57:31,223 - INFO - replace empty position_groups with position info...
2023-07-16 17:57:31,237 - INFO - replace empty player_name with player_display_name info...
2023-07-16 17:57:31,249 - INFO - replace empty headshot_url with 'none'...
2023-07-16 17:57:31,259 - INFO - fillna(0) for all binary columns...
2023-07-16 17:57:31,259 - INFO - Impute columns to 0
2023-07-16 17:57:31,452 - WARNING - After merge player_stats count changed - went from 124081 to 124083


CPU times: user 331 ms, sys: 62.3 ms, total: 394 ms
Wall time: 343 ms


---

# <font color=teal>direct loads </font>

### <font color="#9370DB">adv stats</font>

In [11]:
%%time

advstats_def_df = load_files('advstats-season-def')
advstats_pass_df = load_files('advstats-season-pass')
advstats_rec_df = load_files('advstats-season-rec')
advstats_rush_df = load_files('advstats-season-rush')


2023-07-16 17:57:31,460 - INFO - Reading all files from advstats-season-def
2023-07-16 17:57:31,461 - INFO -   + Reading advstats-season-def.parquet
2023-07-16 17:57:31,466 - INFO - Reading all files from advstats-season-pass
2023-07-16 17:57:31,467 - INFO -   + Reading advstats-season-pass.parquet
2023-07-16 17:57:31,470 - INFO - Reading all files from advstats-season-rec
2023-07-16 17:57:31,470 - INFO -   + Reading advstats-season-rec.parquet
2023-07-16 17:57:31,474 - INFO - Reading all files from advstats-season-rush
2023-07-16 17:57:31,475 - INFO -   + Reading advstats-season-rush.parquet


CPU times: user 17.9 ms, sys: 5.3 ms, total: 23.2 ms
Wall time: 16.7 ms


### <font color="#9370DB">nextgen stats</font>

In [12]:
%%time
next_pass_df = load_files('nextgen-passing')


2023-07-16 17:57:31,480 - INFO - Reading all files from nextgen-passing
2023-07-16 17:57:31,481 - INFO -   + Reading nextgen-passing_2017.csv.gz
2023-07-16 17:57:31,485 - INFO -   + Reading nextgen-passing_2021.csv.gz
2023-07-16 17:57:31,491 - INFO -   + Reading nextgen-passing_2019.csv.gz
2023-07-16 17:57:31,495 - INFO -   + Reading nextgen-passing_2020.csv.gz
2023-07-16 17:57:31,499 - INFO -   + Reading nextgen-passing_2016.csv.gz
2023-07-16 17:57:31,503 - INFO -   + Reading nextgen-passing_2022.csv.gz
2023-07-16 17:57:31,507 - INFO -   + Reading nextgen-passing_2018.csv.gz


CPU times: user 25.9 ms, sys: 4.4 ms, total: 30.3 ms
Wall time: 32.9 ms


In [13]:
%%time
next_rec_df = load_files('nextgen-receiving')


2023-07-16 17:57:31,515 - INFO - Reading all files from nextgen-receiving
2023-07-16 17:57:31,516 - INFO -   + Reading nextgen-receiving_2021.csv.gz
2023-07-16 17:57:31,522 - INFO -   + Reading nextgen-receiving_2017.csv.gz
2023-07-16 17:57:31,529 - INFO -   + Reading nextgen-receiving_2019.csv.gz
2023-07-16 17:57:31,535 - INFO -   + Reading nextgen-receiving_2016.csv.gz
2023-07-16 17:57:31,541 - INFO -   + Reading nextgen-receiving_2020.csv.gz
2023-07-16 17:57:31,548 - INFO -   + Reading nextgen-receiving_2018.csv.gz
2023-07-16 17:57:31,554 - INFO -   + Reading nextgen-receiving_2022.csv.gz


CPU times: user 38.1 ms, sys: 7.13 ms, total: 45.3 ms
Wall time: 47.6 ms


In [14]:
%%time
next_rush_df = load_files('nextgen-rushing')

2023-07-16 17:57:31,568 - INFO - Reading all files from nextgen-rushing
2023-07-16 17:57:31,569 - INFO -   + Reading nextgen-rushing_2018.csv.gz
2023-07-16 17:57:31,573 - INFO -   + Reading nextgen-rushing_2022.csv.gz
2023-07-16 17:57:31,576 - INFO -   + Reading nextgen-rushing_2016.csv.gz
2023-07-16 17:57:31,579 - INFO -   + Reading nextgen-rushing_2020.csv.gz
2023-07-16 17:57:31,582 - INFO -   + Reading nextgen-rushing_2019.csv.gz
2023-07-16 17:57:31,585 - INFO -   + Reading nextgen-rushing_2021.csv.gz
2023-07-16 17:57:31,588 - INFO -   + Reading nextgen-rushing_2017.csv.gz


CPU times: user 19.9 ms, sys: 3.33 ms, total: 23.2 ms
Wall time: 24.1 ms


### <font color="#9370DB">players</font>

In [15]:
%%time
players_df = load_files('players')
players_df = transform_players(players_df)

2023-07-16 17:57:31,606 - INFO - Reading all files from players
2023-07-16 17:57:31,607 - INFO -   + Reading players.parquet
2023-07-16 17:57:31,641 - INFO - Process players dataset...
2023-07-16 17:57:31,642 - INFO - drop players without gsis_ids - they won't link to player_stats
2023-07-16 17:57:31,657 - INFO - fill empty players status to 'NONE'
2023-07-16 17:57:31,663 - INFO - rename gsis_id to player_id...


CPU times: user 56.2 ms, sys: 9.32 ms, total: 65.5 ms
Wall time: 59.1 ms


---

# <font color=teal>store to database so we can perform some SQL operations</font>

In [16]:
def load_all_datasets_to_db(data: dict):
    data['schema'] = database_schema
    load_dims_to_db(data)


In [17]:
%%time
if LOAD_TO_DB:
    datasets.update({
        'players': players_df,
        'player_stats': stats_df,
        'adv_stats_def': advstats_def_df,
        'adv_stats_pass': advstats_pass_df,
        'adv_stats_rec': advstats_rec_df,
        'adv_stats_rush': advstats_rush_df,
        'nextgen_pass': next_pass_df,
        'nextgen_rec': next_rec_df,
        'nextgen_rush': next_rush_df
    })
    load_all_datasets_to_db(datasets)

2023-07-16 18:01:50,745 - INFO - create table play_actions in schema controls
2023-07-16 18:02:30,364 - INFO - create table game_drive in schema controls
2023-07-16 18:02:50,642 - INFO - create table play_analytics in schema controls
2023-07-16 18:04:07,652 - INFO - create table play_situations in schema controls
2023-07-16 18:04:35,098 - INFO - create table play_metrics in schema controls
2023-07-16 18:04:56,809 - INFO - create table player_events in schema controls
2023-07-16 18:05:06,916 - INFO - create table game_info in schema controls
2023-07-16 18:05:07,060 - INFO - create table player_participation in schema controls
2023-07-16 18:09:05,847 - INFO - create table players in schema controls
2023-07-16 18:09:07,370 - INFO - create table player_stats in schema controls
2023-07-16 18:09:23,297 - INFO - create table adv_stats_def in schema controls
2023-07-16 18:09:23,690 - INFO - create table adv_stats_pass in schema controls
2023-07-16 18:09:23,746 - INFO - create table adv_stats_r

CPU times: user 3min 46s, sys: 54.3 s, total: 4min 40s
Wall time: 7min 34s
